In [1]:
import json
import time
from datetime import datetime
import pytz
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

C:\Users\zhuta\Anaconda3\envs\py27\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# all the functions

def load_file(files):
    #info of total tweets
    tw_info_total = []
    for line in files :
        #info of current tweets
        tw_info = []
        curr_tw = line
        #time current tweet is posted
        date = curr_tw['citation_date']
        #follower of this user
        foll = curr_tw['author']['followers']
        #name of author
        author = curr_tw['author']['name']
        #length of tweet
        len_tweet = len(curr_tw['tweet']['text'])
        #favorite count
        favor_ct = curr_tw['tweet']['favorite_count']
        #user mentioned count
        user_ment_ct = len(curr_tw['tweet']['entities']['user_mentions'])
        #store info as (date,author,follower,retweet_count)
        tw_info.append(date)
        tw_info.append(author)
        tw_info.append(foll)
        tw_info.append(len_tweet)
        tw_info.append(favor_ct)
        tw_info.append(user_ment_ct)
        tw_info_total.append(tw_info)
    #convert to dataframe
    df = pd.DataFrame(tw_info_total,columns=['time','author','follower',
                                             'len of tweet','favorite_count','number of user_mentioned'])
    df = df.sort_values(by = 'time')
    df = df.reset_index(drop=True)
    
    return df

def get_hour(time_stamp):
    pst_tz = pytz.timezone('US/Pacific')
    return (datetime.fromtimestamp(time_stamp, pst_tz)).hour

def avg_rmse_lr(features,labels):
    kf = KFold(n_splits=10, random_state=0, shuffle=True)

    test_mse_list = []

    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
    
        linearregression.fit(X_train, y_train)
        
        # Compute mse for test set
        y_test_pred = linearregression.predict(X_test)
        test_mse_list.append(mean_squared_error(y_test, y_test_pred))
    
    avg_test_rmse = np.sqrt(np.mean(test_mse_list))

    print ('RMSE for linear regression model is', avg_test_rmse)

def avg_rmse_rf(features,labels):
    kf = KFold(n_splits=10, random_state=0, shuffle=True)

    test_mse_list = []

    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
    
        rf_regressor.fit(X_train, y_train)
        
        # Compute mse for test set
        y_test_pred = rf_regressor.predict(X_test)
        test_mse_list.append(mean_squared_error(y_test, y_test_pred))
    
    avg_test_rmse = np.sqrt(np.mean(test_mse_list))

    print ('RMSE for Random Forest Regressor model is', avg_test_rmse)

def avg_rmse_svm(features,labels):
    kf = KFold(n_splits=10, random_state=0, shuffle=True)

    test_mse_list = []

    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
    
        svm.fit(X_train, y_train)
        
        # Compute mse for test set
        y_test_pred = svm.predict(X_test)
        test_mse_list.append(mean_squared_error(y_test, y_test_pred))
    
    avg_test_rmse = np.sqrt(np.mean(test_mse_list))

    print ('RMSE for LinearSVR model is', avg_test_rmse)
    
    
def get_features(df):
    tw_tot = 0
    foll_tot = 0
    author_visited_dict = dict()
    len_tweet = 0
    favorite_ct = 0
    user_ment = 0
    start_hour = get_hour(df.iloc[0,0])
    
    feature_list_tot = []
    feature_list_curr = []
    
            
    for index, row in df.iterrows():
        curr_hour = get_hour(row['time'])
        curr_author = row['author']
        curr_foll = row['follower'] 
        curr_len_tweet = row['len of tweet'] 
        curr_favorite_ct = row['favorite_count']
        curr_user_ment = row['number of user_mentioned']
        
        if curr_hour == start_hour:
            tw_tot += 1
            len_tweet += curr_len_tweet
            favorite_ct += curr_favorite_ct
            user_ment += curr_user_ment
            if (curr_author not in author_visited_dict):
                foll_tot += curr_foll
                author_visited_dict[curr_author] = True;
            
        
        else:
            feature_list_curr.append(start_hour)
            feature_list_curr.append(tw_tot)
            feature_list_curr.append(foll_tot)
            feature_list_curr.append(len_tweet/tw_tot)
            feature_list_curr.append(favorite_ct)
            feature_list_curr.append(user_ment)
            feature_list_tot.append(feature_list_curr)
        
            #setup the counters
            tw_tot = 1
            foll_tot = curr_foll
            len_tweet = curr_len_tweet
            favorite_ct = curr_favorite_ct
            user_ment = curr_user_ment
            start_hour = curr_hour
            feature_list_curr = []
            author_visited_dict.clear()
        
    feature_df = pd.DataFrame(feature_list_tot,columns=['time','tweets_total','followers_total',
                                'len of tweet(avg)','favorite_count','number of user_mentioned'])
    return feature_df

def cross_validation_3period(data):
    # split tweets
    time1 = 1422806400 #20150201 8:00am
    time2 = 1422849600 #20150201 8:00pm

    data_1 = []
    data_2 = []
    data_3 = []
    for i in range(0,len(data)):
        tweet = data[i]
        time = tweet["firstpost_date"]
        if   time < time1: 
             data_1.append(tweet)
        elif time >= time1 and time < time2: 
             data_2.append(tweet)
        else: 
             data_3.append(tweet)

    # first period
    df_1 = load_file(data_1)
    feature_df_1 = get_features(df_1)
    y_1 = np.nan_to_num(feature_df_1['tweets_total'].values)
    feature_df_1.drop(columns =['time'],inplace = True)
    x_1 = np.nan_to_num(feature_df_1.values)
    x_1_len = x_1.shape[0]
    train_1 = x_1[0:x_1_len-1,:]
    y_1_len = len(y_1)
    target_1 = y_1[1:y_1_len]

    # rmse_1
    print "first period"
    avg_rmse_lr(train_1,target_1)
    avg_rmse_rf(train_1,target_1)
    avg_rmse_svm(train_1,target_1)
    
    # second period
    df_2 = load_file(data_2)
    feature_df_2 = get_features(df_2)
    y_2 = np.nan_to_num(feature_df_2['tweets_total'].values)
    feature_df_2.drop(columns =['tweets_total'],inplace = True)
    x_2 = np.nan_to_num(feature_df_2.values) 
    x_2_len = x_2.shape[0]
    train_2 = x_2[0:x_2_len-1,:]
    y_2_len = len(y_2)
    target_2 = y_2[1:y_2_len]

    # rmse_2
    print "second period"
    avg_rmse_lr(train_2,target_2)
    avg_rmse_rf(train_2,target_2)
    avg_rmse_svm(train_2,target_2)
    
    
    # third period
    df_3 = load_file(data_3)
    feature_df_3 = get_features(df_3)
    y_3 = np.nan_to_num(feature_df_3['tweets_total'].values)
    feature_df_3.drop(columns =['tweets_total'],inplace = True)
    x_3 = np.nan_to_num(feature_df_3.values)
    x_3_len = x_3.shape[0]
    train_3 = x_3[0:x_3_len-1,:]
    y_3_len = len(y_3)
    target_3 = y_3[1:y_3_len]

    # rmse_3
    print "third period"
    avg_rmse_lr(train_3,target_3)
    avg_rmse_rf(train_3,target_3)
    avg_rmse_svm(train_3,target_3)

In [3]:
path = "tweet_data/"

files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", 
        "tweets_#nfl.txt", "tweets_#patriots.txt", 
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]

for index, name in enumerate(files):
    print ("files[" + str(index) + "] => " + name)
    
    


files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


In [4]:
linearregression = LinearRegression()
rf_regressor = RandomForestRegressor(n_estimators=13,
                             max_features=3,
                             max_depth=11,
                             bootstrap=True,
                             oob_score=True,
                             random_state=0)
svm = LinearSVR(random_state = 0)

# Problem 1.4 (i)

In [7]:
# hashtag tweets_#gohawks 

# load #gohawks
f = open('tweet_data/tweets_#gohawks.txt')
f_start = f.tell()
f.seek(f_start)
gohawks = []

for line in f.readlines():
    tweet = json.loads(line)
    gohawks.append(tweet)

print 'hashtag gohawks'
cross_validation_3period(gohawks)

hashtag gohawks
first period
('RMSE for linear regression model is', 1655.7874532940464)


C:\Users\zhuta\Anaconda3\envs\py27\lib\site-packages\sklearn\ensemble\forest.py:724: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


('RMSE for Random Forest Regressor model is', 1055.2969843204903)
('RMSE for LinearSVR model is', 1562.3615132541765)
second period
('RMSE for linear regression model is', 3683.0478759575235)
('RMSE for Random Forest Regressor model is', 2179.7317581245466)
('RMSE for LinearSVR model is', 2924.312014044475)
third period
('RMSE for linear regression model is', 615.5892138852992)
('RMSE for Random Forest Regressor model is', 55.99420899546564)
('RMSE for LinearSVR model is', 248.45264214824218)


In [8]:
# hashtag tweets_#gopatriots 
# load #gopatriots
f = open('tweet_data/tweets_#gopatriots.txt')
f_start = f.tell()
f.seek(f_start)
gopatriots = []

for line in f.readlines():
    tweet = json.loads(line)
    gopatriots.append(tweet)

print 'hashtag gopatriots'
cross_validation_3period(gopatriots)  

hashtag gopatriots
first period
('RMSE for linear regression model is', 60.18790319016534)
('RMSE for Random Forest Regressor model is', 68.35493721903643)
('RMSE for LinearSVR model is', 161.74560263742472)
second period
('RMSE for linear regression model is', 657.9309081556767)
('RMSE for Random Forest Regressor model is', 952.5419144152917)
('RMSE for LinearSVR model is', 956.8085099924239)
third period
('RMSE for linear regression model is', 21.633772270996058)
('RMSE for Random Forest Regressor model is', 7.41140772708137)
('RMSE for LinearSVR model is', 93.7721670468381)


In [9]:
# hashtag tweets_#nfl 
# load #nfl
f = open('tweet_data/tweets_#nfl.txt')
f_start = f.tell()
f.seek(f_start)
nfl = []

for line in f.readlines():
    tweet = json.loads(line)
    nfl.append(tweet)

print 'hashtag nfl'
cross_validation_3period(nfl) 

hashtag nfl
first period
('RMSE for linear regression model is', 321.0465218882432)
('RMSE for Random Forest Regressor model is', 305.4140009525108)
('RMSE for LinearSVR model is', 1171.4519112789958)
second period
('RMSE for linear regression model is', 2888.8349056280367)
('RMSE for Random Forest Regressor model is', 2524.5338395047775)
('RMSE for LinearSVR model is', 3386.0441364157714)
third period
('RMSE for linear regression model is', 182.4218811483287)
('RMSE for Random Forest Regressor model is', 174.21483870004218)
('RMSE for LinearSVR model is', 1725.9893112047482)


In [10]:
# hashtag tweets_#patriots 
# load #patriots
f = open('tweet_data/tweets_#patriots.txt')
f_start = f.tell()
f.seek(f_start)
patriots = []

for line in f.readlines():
    tweet = json.loads(line)
    patriots.append(tweet)

print 'hashtag patriots'
cross_validation_3period(patriots) 

hashtag patriots
first period
('RMSE for linear regression model is', 758.9693834063161)
('RMSE for Random Forest Regressor model is', 702.2328131313715)
('RMSE for LinearSVR model is', 1040.2187499858069)
second period
('RMSE for linear regression model is', 39662.249707314775)
('RMSE for Random Forest Regressor model is', 10443.47850871689)
('RMSE for LinearSVR model is', 21010.38378624736)
third period
('RMSE for linear regression model is', 258.28652280381993)
('RMSE for Random Forest Regressor model is', 140.43237396188124)
('RMSE for LinearSVR model is', 3040.162008368268)


In [5]:
# hashtag tweets_#sb49 
# load #sb49
f = open('tweet_data/tweets_#sb49.txt')
f_start = f.tell()
f.seek(f_start)
sb49 = []

for line in f.readlines():
    tweet = json.loads(line)
    sb49.append(tweet)

print 'hashtag sb49'
cross_validation_3period(sb49) 

hashtag sb49
first period
('RMSE for linear regression model is', 207.4425112867843)


C:\Users\zhuta\Anaconda3\envs\py27\lib\site-packages\sklearn\ensemble\forest.py:724: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


('RMSE for Random Forest Regressor model is', 186.0088144246698)
('RMSE for LinearSVR model is', 54735.90870481248)
second period
('RMSE for linear regression model is', 20763.609673674848)
('RMSE for Random Forest Regressor model is', 23268.34219735084)
('RMSE for LinearSVR model is', 117894.2639963747)
third period
('RMSE for linear regression model is', 1378.7780622922753)
('RMSE for Random Forest Regressor model is', 239.0489169702358)
('RMSE for LinearSVR model is', 9761.941331376875)


In [ ]:
# hashtag tweets_#superbowl 
# load #superbowl
f = open('tweet_data/tweets_#superbowl.txt')
f_start = f.tell()
f.seek(f_start)
superbowl = []

for line in f.readlines():
    tweet = json.loads(line)
    superbowl.append(tweet)

print 'hashtag superbowl'
cross_validation_3period(superbowl) 

# (ii）

In [ ]:
# all hashtags 
# By observing the top 3 features of each hashtag, we decided to choose 4 features, excluding 'time', to predict and fit the model
# load #gohawks
allhashtag = ['tweets_#gohawks.txt','tweets_#gopatriots.txt','tweets_#nfl.txt',
              'tweets_#patriots.txt','tweets_#sb49.txt','tweets_#superbowl.txt']

for hashtag in allhashtag:
    f = open ('tweet_data/' + hashtag)
    f_start = f.tell()
    f.seek(f_start)
    all_hashtag = []

    for line in f.readlines():
        tweet = json.loads(line)
        all_hashtag.append(tweet)

print 'all hashtag'
cross_validation_3period(all_hashtag)

# Part 1.5

In [ ]:
# first period testing data
period_1_samples = ['sample1_period1','sample4_period1','sample5_period1','sample8_period1']
period_2_samples = ['sample2_period2','sample6_period2','sample9_period2']
period_3_samples = ['sample3_period3','sample7_period3','sample10_period3']

for sample in period_1_samples
    f = open ('test_data/' + sample)
    f_start = f.tell()
    f.seek(f_start)
    period_1_test = []

    for line in f.readlines():
        tweet = json.loads(line)
        period_1_test.append(tweet)
        
for sample in period_2_samples
    f = open ('test_data/' + sample)
    f_start = f.tell()
    f.seek(f_start)
    period_2_test = []

    for line in f.readlines():
        tweet = json.loads(line)
        period_2_test.append(tweet)
        
for sample in period_3_samples
    f = open ('test_data/' + sample)
    f_start = f.tell()
    f.seek(f_start)
    period_3_test = []

    for line in f.readlines():
        tweet = json.loads(line)
        period_3_test.append(tweet)
        
# split tweets
time1 = 1422806400 #20150201 8:00am
time2 = 1422849600 #20150201 8:00pm

data_1 = []
data_2 = []
data_3 = []
for i in range(0,len(all_hashtag)):
    tweet = all_hashtag[i]
    time = tweet["firstpost_date"]
    if   time < time1: 
         data_1.append(tweet)
    elif time >= time1 and time < time2: 
         data_2.append(tweet)
    else: 
         data_3.append(tweet)

df_1 = load_file(data_1)
feature_df_1 = get_features(df_1)
y_1 = np.nan_to_num(feature_df_1['tweets_total'].values)
feature_df_1.drop(columns =['tweets_total'],inplace = True)
x_1 = np.nan_to_num(feature_df_1.values)
rf_regressor.fit(x_1, y_1)

df_test_1 = load_file(period_1_test)
feature_df_test_1 = get_features(df_test_1)
y_test_1 = np.nan_to_num(feature_df_test_1['tweets_total'].values)
feature_df_test_1.drop(columns =['tweets_total'],inplace = True)
x_test_1 = np.nan_to_num(feature_df_test_1.values)

#calculating rmse for first period test data
y_test_pred_1 = rf_regressor.predict(x_test_1)
rmse_test_1 = np.sqrt(mean_squared_error(y_test_1, y_test_pred))


df_2 = load_file(data_2)
feature_df_2 = get_features(df_2)
y_2 = np.nan_to_num(feature_df_2['tweets_total'].values)
feature_df_2.drop(columns =['tweets_total'],inplace = True)
x_2 = np.nan_to_num(feature_df_2.values)
rf_regressor.fit(x_2, y_2)

df_test_2 = load_file(period_2_test)
feature_df_test_2 = get_features(df_test_2)
y_test_2 = np.nan_to_num(feature_df_test_2['tweets_total'].values)
feature_df_test_2.drop(columns =['tweets_total'],inplace = True)
x_test_2 = np.nan_to_num(feature_df_test_2.values)

#calculating rmse for second period test data
y_test_pred_2 = rf_regressor.predict(x_test_2)
rmse_test_2 = np.sqrt(mean_squared_error(y_test_2, y_test_pred_2))


df_3 = load_file(data_3)
feature_df_3 = get_features(df_3)
y_3 = np.nan_to_num(feature_df_3['tweets_total'].values)
feature_df_3.drop(columns =['tweets_total'],inplace = True)
x_3 = np.nan_to_num(feature_df_3.values)
rf_regressor.fit(x_3, y_3)

df_test_3 = load_file(period_3_test)
feature_df_test_3 = get_features(df_test_3)
y_test_3 = np.nan_to_num(feature_df_test_3['tweets_total'].values)
feature_df_test_3.drop(columns =['tweets_total'],inplace = True)
x_test_3 = np.nan_to_num(feature_df_test_3.values)

#calculating rmse for second period test data
y_test_pred_3 = rf_regressor.predict(x_test_3)
rmse_test_3 = np.sqrt(mean_squared_error(y_test_3, y_test_pred_3))